In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline

In [2]:
def getSignal(events, stepSize, prob, pred, numClasses, numThreads, **kargs):
    #bet sizing signal from predictions
    if prob.shape[0]==0:
        return pd.series()
    #1. Generate signal from multinomial classification (one vs rest (OvR))
    signal0 = (prob-1./numClasses)/(prob*(1.-prob))**.5 #t-value of OvR
    signal0 = pred*(2*norm.cdf(signal0)-1) #signal = side * size
    if 'side' in events:
        signal0*=events.loc[signal0.index, 'side'] #meta labeling
        
    #2. compute avg symbol amongst those concurrently open 
    df0 = signal0.to_frame('signal').join(events[['t1']], how='left')
    df0 = avgActiveSignals(df0, numThreads)
    signal1 = discreteSignal(signal0=df0, stepSize=stepSize)
    return signal1
    

In [3]:
#averaging across active bets
def mpAvgActiveSignals(signals, molecule):
    ''' at time loc, average signal among those still active. signal is active if: 
    1. issued before or at loc and 
    2. loc before signals end time, or endtime still unknown'''
    out = pd.Series()
    for loc in molecule:
        df0 = (signals.index.values<=loc)&((loc<signals['t1'])|pd.isnull(signals['t1']))
        act = signals[df0].index
        if len(act)>0: 
            out[loc] = signals.loc[act, 'signal'].mean()
        else:
            out[loc]= 0 #no active signals
        return out
def avgActiveSignals(signals, numThreads):
    #1. time points where signals change (either one starts or one ends)
    tPnts = set(signals['t1'].dropna().values)
    tPnts = tPnts.union(signals.index.values)
    tPnts = list(tPnts); tPnts.sort()
    out = mpPandasObj(mpAvgActiveSignals, ('molecule',tPnts), numThreads, signals=signals)
    return out 

In [4]:
#size discretization to prevent overtrading
def discreteSignal(signal0, stepSize):
    signal1 = (signal0/stepSize).round()*stepSize
    signal1[signal1>1] = 1 # cap
    signal1[signal1<-1] = -1 #floor
    return signal1

In [8]:
#dynamic position sizing and limit price
def betSize(w, x):
    return x*(w+x**2)**-.5
def getTPos(w,f, mP, maxPos):
    return int(betSize(w,f-mP)*maxPos)
def invPrice(f, w, m):
    return f-m*(w/(1-m**2))**-.5
def limitPrice(tPos, pos, f, w, maxPos):
    sgn = (1 if tPos>=pos else -1)
    lP=0
    for j in range(abs(pos+sgn), abs(tPos+1)):
        lP+=invPrice(f,w,j/float(maxPos))
    lP/=tPos-pos
    return lP
def getW(x, m):
    return x**2*(m**-2-1)
def main():
    pos, maxPos, mP, f, wParams = 0, 100, 100, 115, {'divergence':10, 'm':.95}
    w = getW(wParams['divergence'], wParams['m'])
    tPos = getTPos(w,f, mP, maxPos)
    lP = limitPrice(tPos, pos, f,w,maxPos)
    return
    